In [ ]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from readers.audio import AudioSegment
from readers.featurizer import Wave2Mel
import torch

## 先合并status和statusSSL列

In [ ]:
status_id = []
# status2label = {"healthy": 0, "symptomatic": 1, "COVID-19": 2}
for i, row in enumerate(metadf.itertuples()):
    ano = -1
    sta = getattr(row, "status")
    if str(sta) != "nan":
        status_id.append(int(sta))
    else:
        sta = getattr(row, "status_SSL")
        if str(sta) != "nan":
            status_id.append(int(sta))
        else:
            print(sta)
    # if i%300==0:
        # print(i, ':', getattr(row, "filename"))
print(len(status_id))
metadf["status_full"] = status_id
metadf.head(110)
metadf.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labedfine_staaSSL.csv", sep=',')

# 清洗后分为均匀的分类数据集

In [1]:
# Step1 先统计标签0,1,2的数目
import pandas as pd
metafile = "C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labedfine_staaSSL.csv"
# status_cnt = [0, 0, 0]
heal_indices = []
symp_indices = []
cov19_indices = []
with open(metafile, 'r') as fin:
    fin.readline()
    line = fin.readline()
    ind = 0
    while line:
        parts = line.strip().split(',')
        s = int(parts[7])
        if s == 0:
            # status_cnt[0] += 1
            heal_indices.append(ind)
        elif s== 1:
            # status_cnt[1] += 1
            symp_indices.append(ind)
        elif s==2:
            # status_cnt[2] += 1
            cov19_indices.append(ind)
        else:
            print("Error")
        line = fin.readline()
        ind += 1
# print(status_cnt)
print(len(heal_indices),len(symp_indices),len(cov19_indices))
metadf = pd.read_csv(metafile, delimiter=',', header=0, index_col=0)
metadf.iloc[:, [0, 6]].groupby("status_full").count()

13709 3288 939


,filename
status_full,
0,13709
1,3288
2,939


In [2]:
# Step 2
# 然后，设定标签0的数目为标签1,2的平均值，以保持最大的平衡，抽取出来对应的行，拼接保存
import random
heal_indices = random.choices(heal_indices, k=(3288+939)//2+1)

cnt_list = [(3288+939)//2+1, 3288, 939]
resdf = metadf.iloc[heal_indices, :]
resdf = pd.concat([resdf, metadf.iloc[symp_indices, :]], axis=0)
resdf = pd.concat([resdf, metadf.iloc[cov19_indices, :]], axis=0)
resdf.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labedfine_forcls.csv", sep=',')

In [3]:
# Step 3 测试是否成功
import pandas as pd
metafile = "C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labedfine_forcls.csv"
metadf = pd.read_csv(metafile, delimiter=',', header=0, index_col=0)
metadf.iloc[:, [0, 6]].groupby("status_full").count()

,filename
status_full,
0,2114
1,3288
2,939


# 绘图测试

In [ ]:
k=1
audio_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/001e2f19-d81c-4029-b33c-d2db56b23a4a.webm"
# audioseg = AudioSegment.from_file("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/00422dad-066b-402f-9cac-e9efe51442af.wav")

In [ ]:
audioseg = AudioSegment.from_file(audio_path)
ipd.Audio(audio_path)

In [ ]:
def concatenate(wave, overlap=200):
    total_len = sum([len(x) for x in wave])
    unfolded = np.zeros(total_len)

    # Equal power crossfade
    window = np.hanning(2 * overlap)
    fade_in = window[:overlap]
    fade_out = window[-overlap:]

    end = total_len
    for i in range(1, len(wave)):
        prev = wave[i - 1]
        curr = wave[i]

        if i == 1:
            end = len(prev)
            unfolded[:end] += prev

        max_idx = 0
        max_corr = 0
        pattern = prev[-overlap:]
        # slide the curr batch to match with the pattern of previous one
        for j in range(overlap):
            match = curr[j:j + overlap]
            corr = np.sum(pattern * match) / [(np.sqrt(np.sum(pattern ** 2)) * np.sqrt(np.sum(match ** 2))) + 1e-8]
            if corr > max_corr:
                max_idx = j
                max_corr = corr

        # Apply the gain to the overlap samples
        start = end - overlap
        unfolded[start:end] *= fade_out
        end = start + (len(curr) - max_idx)
        curr[max_idx:max_idx + overlap] *= fade_in
        unfolded[start:end] += curr[max_idx:]
    return unfolded[:end]

In [ ]:
def vad(wav, top_db=40, overlap=200):
    # Split an audio signal into non-silent intervals
    intervals = librosa.effects.split(wav, top_db=top_db)
    if len(intervals) == 0:
        return wav
    wav_output = [np.array([])]
    for sliced in intervals:
        seg = wav[sliced[0]:sliced[1]]
        if len(seg) < 2 * overlap:
            wav_output[-1] = np.concatenate((wav_output[-1], seg))
        else:
            wav_output.append(seg)
    wav_output = [x for x in wav_output if len(x) > 0]

    if len(wav_output) == 1:
        wav_output = wav_output[0]
    else:
        wav_output = concatenate(wav_output)
    return wav_output

def wav_padding(wave, save_len=48000):
    L = wave.shape[0]
    if L >= save_len:
        return
    new_signal = np.zeros(save_len)
    resi = save_len - L
    new_signal[:L] = wave
    # print("resi:", resi)
    interval = 1500
    i = 1
    while resi > 0:
        # print("resi:", resi)
        if resi - interval - L > 0:
            new_signal[i * (L + interval):i * (L + interval) + L] = wave
            resi -= interval + L
        elif resi - interval > 0:
            LL = resi - interval
            new_signal[i * (L + interval):i * (L + interval) + LL] = wave[:LL]
            break
        else:
            break
        i += 1
    return new_signal

wave = audioseg.samples
plt.figure(0)
plt.subplot(3,2,1)
plt.plot(range(len(wave)), wave, c="black")
plt.subplot(3,2, 2)
plt.imshow(w2m(torch.from_numpy(wave)))

wave1 = vad(wave)
plt.subplot(3,2,3)
plt.plot(range(len(wave1)), wave1, c="black")
plt.subplot(3,2, 4)
plt.imshow(w2m(torch.from_numpy(wave1).to(torch.float32)))

wave2 = wav_padding(wave1)
plt.subplot(3,2,5)
plt.plot(range(len(wave2)), wave2, c="black")
plt.subplot(3,2, 6)
plt.imshow(w2m(torch.from_numpy(wave2).to(torch.float32)))

plt.show()

In [ ]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/')
# sys.path.append(r'D:/PythonTorchVITS/CoughDL/SoundDL-CoughVID/')
import os
import numpy as np
import librosa
import pandas as pd
import IPython.display as ipd
import matplotlib.pyplot as plt
from readers.audio import AudioSegment
# root_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/"

In [ ]:
root_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/"
metafile = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv"
metadf = pd.read_csv(metafile, delimiter=',', header=0, index_col=0)

# 去除静音片段然后计算时长然后保存标签

In [ ]:
metadf = pd.read_csv(metafile, delimiter=',', header=0, index_col=0)
# print(metadf.head(5))

fname_list = []
dura_list = []
anomaly_list = []
status_id = []
status_SSL_id = []
status2label = {"healthy": 0, "symptomatic": 1, "COVID-19": 2}
for i, row in enumerate(metadf.itertuples()):
    wi = getattr(row, "uuid")
    fname = root_path + wi
    for ext in ["webm", "wav", "ogg"]:
        if os.path.exists(fname+'.'+ext):
            fname = fname + '.' + ext
    fname_list.append(fname)
    try:
        audioseg = AudioSegment.from_file(file=fname)
    except Exception as e:
        print(e)
        print("Read Error:", fname, "skip it and continue.")
    audioseg.vad(top_db=40)
    dura_list.append(audioseg.duration)
    ano = -1
    sta_str = getattr(row, "status")
    if isinstance(sta_str, str):
        sid = status2label[sta_str]
        status_id.append(sid)
        if sid == 0:
            ano = 0
        else:
            ano = 1
    else:
        status_id.append(sta_str)
    stassl_str = getattr(row, "status_SSL")
    if isinstance(stassl_str, str):
        sid = status2label[stassl_str]
        status_SSL_id.append(sid)
        if ano == -1:
            if sid == 0:
                ano = 0
            else:
                ano = 1
    else:
        status_SSL_id.append(stassl_str)
    if ano == -1:
        ano = float('nan')
    anomaly_list.append(ano)
    if i%300==0:
        print(i, ':', fname)
    # if i==19:
    #     break


In [ ]:
wavedura = metadf.iloc[:, [0,2]]  # ,9,10]]
wavedura["uuid"] = fname_list
wavedura["duration"] = dura_list
wavedura["status_id"] = status_id
wavedura["status_SSL_id"] = status_SSL_id
wavedura["anomaly"] = anomaly_list
wavedura.columns = ["filename", "cough_detected", "duration", "status", "status_SSL", "anomaly"]
print(wavedura.head(20))
print(len(wavedura))  # .head(20)

In [ ]:
wavedura.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo.csv", sep=',')

# 把具有异常标注的音频保存下来

In [ ]:
waveinfo_labeled_indices = []
waveinfo_unlabeled_indices = []
i=0
with open("./datasets/waveinfo.csv", 'r') as fin:
    fin.readline()
    line = fin.readline().strip()
    while line:
        parts = line.split(',')
        # print(parts)
        wi = parts[6]
        print(wi, end=', ')
        if len(wi) > 0:
            # waveinfo_labeled_indices.append(parts[0])
            waveinfo_labeled_indices.append(i)
        else:
            # waveinfo_unlabeled_indices.append(parts[0])
            waveinfo_unlabeled_indices.append(i)
        # if i > 40:
        #     break
        i += 1
        line = fin.readline().strip()
# print()
# print(waveinfo_labeled_indices)
# print(waveinfo_unlabeled_indices)

In [ ]:
metadf = pd.read_csv("./datasets/waveinfo.csv", delimiter=',', header=0, index_col=0)
waveinfo_labeled = metadf.iloc[waveinfo_labeled_indices, :]
waveinfo_unlabeled = metadf.iloc[waveinfo_unlabeled_indices, :]
# waveinfo_unlabeled = waveinfo_unlabeled[waveinfo_unlabeled["anomaly"]!=0]
print(len(waveinfo_labeled))
print(len(waveinfo_unlabeled))

In [ ]:
waveinfo_labeled.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled.csv", sep=',')
waveinfo_unlabeled.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_unlabeled.csv", sep=',')

# 去除不符合条件的音频

In [ ]:
datadf1 = pd.read_csv("./datasets/waveinfo_labeled.csv", delimiter=',', header=0, index_col=0)
print(len(datadf1))
# datadf2 = datadf1.where(datadf1["duration"] > 0.36).where(datadf1["duration"] < 13).where(datadf1["cough_detected"]>0.35).dropna(axis=0)
datadf1 = datadf1.where(datadf1["duration"] > 0.36).where(datadf1["duration"] < 13).where(datadf1["cough_detected"]>0.35).dropna(axis=0, subset=["duration"])
print(len(datadf1))

datadf2 = pd.read_csv("./datasets/waveinfo_unlabeled.csv", delimiter=',', header=0, index_col=0)
print(len(datadf2))
# datadf2 = datadf1.where(datadf1["duration"] > 0.36).where(datadf1["duration"] < 13).where(datadf1["cough_detected"]>0.35).dropna(axis=0)
datadf2 = datadf2.where(datadf2["duration"] > 0.36).where(datadf2["duration"] < 13).where(datadf2["cough_detected"]>0.35).dropna(axis=0, subset=["duration"])
print(len(datadf2))

In [ ]:
datadf1.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled_fine.csv", sep=',')
datadf2.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_unlabeled_fine.csv", sep=',')

## 统计每个标签的数目

In [ ]:
import pandas as pd
datadf1 = pd.read_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled_fine.csv", delimiter=',', header=0, index_col=0)

print(datadf1.groupby("status")["filename"].count()）
datadf2 = datadf1.where(pd.isna(datadf1["status"]))
print(datadf2.groupby("status_SSL")["filename"].count())

In [ ]:
import pandas as pd
datadf1 = pd.read_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled_fine.csv", delimiter=',', header=0, index_col=0)
print(datadf1.dropna(subset=["status"]).count())
print(datadf1.dropna(subset=["status_SSL"]).count())

In [ ]:
import pandas as pd
datadf1 = pd.read_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled_fine.csv", delimiter=',', header=0, index_col=0)
datadf1["equal"] = datadf1["status"] == datadf1["status_SSL"]

In [ ]:
print(datadf1.dropna(subset=["status", "status_SSL"]))

In [ ]:
datadf2 = datadf1.dropna(subset=["status", "status_SSL"])
print(datadf2["equal"].count())
print(datadf2.where(datadf1["equal"]==True).count())
print(datadf2.where(datadf1["equal"]==False).count())

# 加上扩展名

In [ ]:

filename = []
for i, uuid in enumerate(datadf1.iloc[:, 0]):
    for ext in ["webm", "wav", "ogg"]:
        fname = root_path + uuid + '.' + ext
        if os.path.exists(fname):
            filename.append(fname)
print("end")

datadf1["filename"] = filename
datadf2 = datadf1.iloc[:, [5,1,2,3,4]]
datadf2.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

# 合并新的列 已完成

In [ ]:
datadf1 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", delimiter=',', header=0, index_col=0)
datadf2 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
# print(datadf1.head(15))
print("row num:", len(datadf1), len(datadf2))
datadf2 = datadf2.iloc[:, [0,9]]
datadf1 = datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)

In [ ]:
datadf3 = datadf1.set_index("uuid").join(datadf2.set_index("uuid"), on="uuid", how="left")
print(len(datadf3))
datadf3

In [ ]:
datadf3.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

## 排序和where筛选

In [ ]:
datadf1 = datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)
datadf1 = datadf1.sort_values("uuid", ascending=False)
datadf1.head(15)

# 波形图绘图

In [ ]:
x, sr = librosa.load(root_path + "ffe8b243-9c7b-49f1-8d7d-a953f736ea4b.wav")
print("sample rate:", sr)
# print("filename:", root_path+df.iloc[0, :][0])
# ipd.Audio(x, rate=sr)

In [ ]:
plt.figure(0)
plt.plot(range(len(x)), x, color="black")
plt.show()

## 读取部分列

In [ ]:
metadata = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
datadf1 = metadata.iloc[:, [0,2,9]]
datadf1 = datadf1.sort_values("status", ascending=True)
datadf1.head(15)

## 已经合并得到的信息↓，不再需要运行

In [ ]:
df = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/dura_stat.csv", delimiter='\t', header=None, index_col=None)
df.columns = ["filename", "nframes", "duration"]
df = df.sort_values("nframes", ascending=True)
print(df.head(15))

In [ ]:
metadata = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
datadf1 = metadata.iloc[:, [0,2]]
datadf1 = datadf1.sort_values("uuid", ascending=True)
datadf1.head(15)

datadf2 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/dura_stat.csv", delimiter='\t', header=None, index_col=None)
datadf2.columns = ["uuid", "nframes", "duration"]
datadf2 = datadf2.sort_values("uuid", ascending=True)
datadf2.head(15)

datadf1["nframes"] = datadf2["nframes"]
datadf1["duration"] = datadf2["duration"]


In [ ]:
datadf1 = datadf1.sort_values("cough_detected", ascending=True)
datadf1.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

In [ ]:
x, sr = librosa.load(root_path + "00bfe21c-ab71-4e5a-a941-4f83f5de5c82.wav")
print("sample rate:", sr)
print("filename:", root_path+df.iloc[0, :][0])
ipd.Audio(x, rate=sr)

In [ ]:
datadf1 = datadf1.sort_values("duration", ascending=False)
datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(0)
plt.scatter(datadf1["cough_detected"], datadf2["duration"])
plt.show()